# 0. Préparation

In [ ]:
import json
import math
import matplotlib.pylab as plb
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import os
import pandas as pd
import statistics as stat
import urllib.request

from collections import Counter
from difflib import SequenceMatcher
from google.colab import drive
from matplotlib.pyplot import figure
from urllib.request import urlopen
from zipfile import ZipFile

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
CHEMIN = 'drive/My Drive/Colab Notebooks/'


# 1. Exploration du jeu de données





## 1.1 Enjeux

Une start-up de la EdTech propose des formations en ligne pour un public de niveau lycée et université.

Cette start-up souhaite s'exporter à l'international :


*   Quels sont les pays avec un fort potentiel de clients pour nos services ?
*   Pour chacun de ces pays, quelle sera l'évolution de ce potentiel de clients ?
*   Dans quels pays l'entreprise doit-elle opérer en priorité ?





## 1.2 Problématique

La start-up a repéré un jeu de données qui pourrait l'intéresser.
La mission est de déterminer si ce jeu de données peut aider le projet d'expansion.

## 1.3 Présentation du jeu de données

### 1.3.0 Préparation

Création des DataFrames

In [ ]:
df_names = ['EdStatsData.csv',
            'EdStatsCountry.csv',
            'EdStatsSeries.csv',
            'EdStatsCountry-Series.csv',
            'EdStatsFootNote.csv']
df_list = [pd.read_csv(CHEMIN + element) for element in df_names]

Affichage du nombre de lignes et de colonnes pour chaque DataFrame

In [ ]:
for i, df in enumerate(df_list):
    temp_string = 'DataFrame \'{}\': {} lines, {} columns.'.format(df_names[i], df.shape[0], df.shape[1])
    print(temp_string + '\n')

### 1.3.1 Fichier Data

Le fichier **EdStatsData** est le fichier le plus volumineux. Il contient, pour chaque pays, les valeurs des indicateurs par année. Le DataFrame contient 886930 lignes and 70 colonnes.

In [ ]:
df_list[0].head()

### 1.3.2 Fichier Country

Le fichier **EdStatsCountry** contient des informations générales sur les pays, leur niveau de richesse, les dates des dernières enquêtes, les catégories financières et politiques, … Il ne contient pas d’indicateur.

In [ ]:
df_list[1].head()

### 1.3.3 Fichier Series

Le fichier EdStatsSeries est le tableau des indicateurs, avec un certain nombre d'informations : déinition, unité de mesure, périodicité, méthode d'agrégation.

In [ ]:
df_list[2].head()

In [ ]:
df_series.dtypes

### 1.3.4 Fichier Country-Series

Le fichier Country Series indique la source de calcul des indicateurs nationaux.

In [ ]:
df_list[3].head()

### 1.3.5 Fichier Footnote

Le fichier Footnote donne la méthode de collecte des indicateurs, selon l'année

In [ ]:
df_list[4].head()

# 2. Analyse pré-exploratoire

## 2.1 Valider la qualité du jeu de données

### 2.1.0 Affichage des valeurs non nulles

Proportion des valeurs NaN dans chaque jeu de données

In [ ]:
def nan_proportion_piechart(df, df_name):
    """Piechart of the nan propotion in a dataframe"""
    cells_nb = df.shape[0] * df.shape[1]
    non_nan_total=0
    for column in df.columns:
        others += df[column].count()
    nan_total = cells_nb - non_nan_total
    values = [nan_total, non_nan_total]
    indices = ['NaN values', ' Other values']
    plt.title('{}, {} elements'.format(df_name, cells_nb))
    plt.pie(values, labels=indices, colors=['red', 'green'])

In [ ]:
for i, df in enumerate(df_list):
    plt.figure(nan_proportion_piechart(df, df_list[i]))

### 2.1.1 Fichier Data

In [ ]:
#Estimation du nombre de données non nulles par année
def nan_values_evolution(df=df_list[0])
    df_years = df.iloc[:, 4:-1]
    values = [df_years[year].isna().sum()/ len(df_years[year]) for year in df_years.columns]
    years = df_years.columns
    plt.title('Proportion des valeurs NaN selon l\'année')
    plt.rcParams["figure.figsize"] = (50,15)
    plt.xticks(rotation=45)
    plt.grid()
    plt.plot(years, values, color='black')

Le nombre de valeurs non nulles par année n'est pas constant, et révèle un tendance générale d'une part, et d'autre part des tendances spécifiques.



*   La valeur moyenne de NaN est de 90% pour les années avant 2017, 95% pour les prévisions (2020 - 2100).
*   La tendance générale montre une diminution des NaN de 1989 à 2015, puis une brusque augmentation.
*   L'année la plus vide est 2017 (100% de valeurs NaN)
*   L'année la plus remplie est 2010 (moins de 75% de NaN).
*   Quelques pics sont bien visibles : de 1975 à 2010, tous les 5 ans, on remarque une diminution subite du nombre de NaN, autrement dit, un pic de données.
*   On peut donc retenir 3 années pour lesquelles les NaN sont les moins nombreuses : 2000, 2005 et 2010.

### 2.1.2 Fichier Country

In [ ]:
df_list[1].info()

Estimation des valeurs NaN de la colonne Currency.

In [ ]:
nb_countries = df_list[1]['Short Name'].count()
nb_currencies = df_list[1]['Currency Unit'].count()
print('Il y a {} valeurs de devises nulles pour {} pays.'.format(nb_countries - nb_currencies, nb_countries))

In [ ]:
# Affichage des pays pour lesquels la devise est nulle
df_list[1][df_list[1]['Currency Unit'].isnull()]['Short Name']

Conclusion: les "pays" n'ayant pas de devise correspondent en réalité aux différentes régions du monde. Une exception cependant, Nauru (Nauru utilise pourtant les dollars australiens, d'après Wikipédia).

### 2.1.3 Fichier Series

In [ ]:
df_list[2].info()

### 2.1.4 Fichier Country-Series

In [ ]:
df_list[3].info()

Estimation des doublons les plus fréquents dans la colonne des indicateurs.

In [ ]:
Counter(df_list[3]['SeriesCode'])

Les indicateurs qui reviennent le plus souvent concernent la population et le taux de croissance : SP.POP.GROW et SP.POP.TOTL

Estimation des doublons les plus fréquents dans la colonne DESCRIPTION

In [ ]:
Counter(df_list[3]['DESCRIPTION']).most_common(10)

La source principale semble être "United Nations World Population Prospects".

### 2.1.5 Fichier Footnote

In [ ]:
df_list[4].info()

Estimation des doublons les plus fréquents de la colonne DESCRIPTION

In [ ]:
Counter(df_list[4]['DESCRIPTION']).most_common(10)

Nombre d'indicateurs par date

In [ ]:
def indicators_evolution(df=df_list[4], year_column='Year'):
    year_counter = sorted(Counter(df[year_column]).items())
    x, y = zip(*year_counter)
    plb.title('Nombre d\'indicateurs par date dans Footnote')
    plt.xticks(rotation=45)
    plb.grid(color='grey', linestyle='-', linewidth=1)
    plb.plot(x, y)

indicators_evolution()

Les indicateurs sont particulièrement nombreux entre les années 1998 et 2014.

Dans l'analyse, il sera intéressant de se concentrer sur les années 1998 à 2014. Ce sont les années où la source de l'indicateur est connue.

## 2.2 Sélection des indicateurs

Un rapide parcours des DataFrame m'a amené à classer les données qui nous intéressent selon les catégories ci-dessous :

1.   Nos **clients** : leur âge, leur niveau d'étude, le taux de chômage, le taux de croissance, ...

2.   Leur capacité à avoir **accès aux formations** : l'alphabétisation, l'accès à internet, ...

3.   Leur **pouvoir d'achat** : pays pauvre, pays riche, ...

4. Les **conditions** dans lesquelles la start-up va opérer : le système économique, les éventuelles aides internationales, la stabilité politique du pays, ...

5. La **fiabilité** des données : l'ancienneté des enquêtes, recensement, ...


### 2.2.1 Repérage des indicateurs

Dans le fichier Series, balayer un par un les 3665 indicateurs est fatigant et vite décourageant.
Après quelques recherches, on s'aperçoit que la colonne 'Topic' contient les **catégories** d'indicateurs, et ne contient que 37 valeurs. On va donc chercher dans un premier temps les catégories qui intéressantes, puis dans un deuxième temps les indicateurs intéressants.

Un exemple de recherche ci-dessous

In [ ]:
# Lister les catégories d'indicateurs
df_series['Topic'].unique()

In [ ]:
df_series[df_series['Topic'] == 'Infrastructure: Communications']['Indicator Name'].unique()

Dans le fichier Series

In [ ]:
series_indic_df = pd.DataFrame({'Catégorie': ['Clients',
                                              'Clients',
                                              'Clients',
                                              'Clients',
                                              'Clients',
                                              'Clients',
                                              'Accès aux formations',
                                              'Accès aux formations',
                                              'Accès aux formations',
                                              'Accès aux formations'],
                                'Indicateur': ['Population, ages 10-18, total',
                                               'Population, ages 15-24, total',
                                               'Gross enrolment ratio, primary to tertiary, gender parity index (GPI)',
                                               'Enrolment in tertiary education per 100,000 inhabitants, both sexes',
                                               'Unemployment, total (% of total labor force) (modeled ILO estimate)',
                                               'Population growth (annual %)',
                                               'Youth literacy rate, population 15-24 years, both sexes (%)',
                                               'Personal computers (per 100 people)',
                                               'Internet users (per 100 people)',
                                               'Mortality rate, under-5 (per 1,000 live births)']})
series_indic_df

Dans le fichier Country

In [ ]:
country_indic_df = pd.DataFrame({'Catégorie': ['Pouvoir d\'achat',
                                               'Conditions d\'activité',
                                               'Conditions d\'activité',
                                               'Conditions d\'activité',
                                               'Conditions d\'activité'],
                                 'Indicateur': ['Income Group',
                                                'System of trade',
                                                'Lending category',
                                                'System of National Accounts',
                                                'Source of most recent Income and expenditure data']})
country_indic_df

Dans le fichier Series

In [ ]:
series_eval_df = pd.DataFrame({'Catégorie': ['Fiabilité des données',
                                             'Fiabilité des données',
                                             'Fiabilité des données',
                                             'Fiabilité des données'],
                               'Indicateur': ['Periodicity',
                                              'Aggregation method',
                                              'Development relevance',
                                              'License Type']})
series_eval_df

Cette DataFrame ne permet pas d'évaluer directement les données que nous cherchons, mais d'évaluer la fiabilité des indicateurs du fichier Series, en donnant la période de collecte, la méthode d'agrégation, ...

Dans le fichier Country-Series

In [ ]:
country_series_eval_df = pd.DataFrame({'Catégorie': ['Fiabilité des données'],
                                    'Indicateur': ['DESCRIPTION']})
country_series_eval_df

### 2.2.2 Evaluation de la fiabilité des indicateurs

Pour l'instant, nous avons juste repéré ces indicateurs car leur intitulé semblait utile, mais nous ne connaissons pas leur proportion de NaN, leur fiabilité, ...

In [ ]:
df_data.isnull().values.sum()

In [ ]:
temp_list = []
for indicator in series_indic_df['Indicateur']:
    temp_df = df_data[df_data['Indicator Name'] == indicator]
    temp_list.append(temp_df.isnull().values.sum() / (temp_df.shape[0] * temp_df.shape[1]))
series_indic_df['Nan Proportion'] = temp_list
series_indic_df.sort_values(by='Nan Proportion', ascending=False)

Les indicateurs 'Unemployment, total (% of total labor force) ...' et 'Mortality rate, under-5 (per 1,000 live births)' ne sont pas utiles, nous pouvons les enlever de notre étude.

In [ ]:
series_indic_df = pd.DataFrame({'Catégorie': ['Clients',
                                              'Clients',
                                              'Clients',
                                              'Clients',
                                              'Clients',
                                              'Accès aux formations',
                                              'Accès aux formations',
                                              'Accès aux formations'],
                                'Indicateur': ['Population, ages 10-18, total',
                                               'Population, ages 15-24, total',
                                               'Gross enrolment ratio, primary to tertiary, gender parity index (GPI)',
                                               'Enrolment in tertiary education per 100,000 inhabitants, both sexes',
                                               'Population growth (annual %)',
                                               'Youth literacy rate, population 15-24 years, both sexes (%)',
                                               'Personal computers (per 100 people)',
                                               'Internet users (per 100 people)']})

Evaluons la fiabilité des indicateurs du fichier Series, afin de rejeter éventuellement ceux qui seraient trop peu fiables.

Création d'un DataFrame rassemblant les indicateurs et les méthodes utilisées pour les évaluer.

In [ ]:
def evaluations_dataframe():
    df_eval = pd.DataFrame({"Catégorie":[],
                            "Indicateur":[],
                            "Evaluateur":[],
                            "Valeur":[]})
    for indicator in series_indic_df['Indicateur']:
        for evaluator in series_eval_df['Indicateur']:
            # Ne récupérer que la valeur cherchée
            temp_series = df_series[df_series['Indicator Name'] == indicator][evaluator]
            for element in temp_series:
                cat_series = series_indic_df[series_indic_df['Indicateur'] == indicator]['Catégorie']
                temp_df = pd.DataFrame({"Catégorie":cat_series,
                                        "Indicateur":[indicator],
                                        "Evaluateur":[evaluator],
                                        "Valeur":[element]})
                df_eval = df_eval.append(temp_df)
    return df_eval

evaluations_dataframe().head(10)

Aperçu des indicateurs pour lesquels l'évaluateur n'est pas un NaN.

In [ ]:
df_eval[df_eval.Valeur.notnull()]

Analyse : deux évaluateurs sont tout le temps vides, 'Development relevance' et 'License Type'. Nous ne pouvons pas nous en servir, nous les enlevons donc de la liste des évaluateurs.

In [ ]:
series_eval = {'Fiabilité des données' : ['Periodicity',
                                          'Aggregation method']}

## 2.3 Ordre de grandeur des indicateurs statistiques

Définissons une fonction qui nous permette d'ordonner une liste des **pays** d'une **région** selon un **indicateur** donné.

In [ ]:
def indic_per_country(region, indic, year):
    temp_df = df_data.merge(df_country[['Country Code', 'Region']],
                            left_on='Country Code',
                            right_on='Country Code')
    temp_df = temp_df[temp_df['Region'] == region]
    temp_df = temp_df[temp_df['Indicator Name'] == indic]
    temp_df = temp_df[['Country Name', year]]
    temp_df = temp_df.sort_values(by=year, ascending=False)
    return temp_df

### 2.3.1 Nos clients

Cherchons les régions du monde satisfaisant le plus possible les indicateurs retenus.

In [ ]:
def get_indics(country, year, category):
    """year must be a string"""
    # Extraction des lignes contenant les indicateurs intéressants
    temp_series = series_indic_df[series_indic_df['Catégorie'] == category]['Indicateur']
    temp_df = df_data[df_data['Indicator Name'].isin(temp_series)]
    temp_df = temp_df[['Country Name', 'Indicator Name', year]]
    temp_df = temp_df[temp_df['Country Name'] == country]
    temp_df = temp_df[temp_df[year].notnull()]
    return temp_df

In [ ]:
get_indics('Europe & Central Asia', '2010', 'Clients')

In [ ]:
df_country['Region'].unique()

In [ ]:
for indicateur in series_indic_df['Indicateur']:
    print('-'*60)
    print(indicateur)
    print(indic_per_country('Europe & Central Asia', indicateur, '2010').head(10))

Les indicateurs 'Youth literacy rate' et 'Personal computers (per 100 people)' contiennent trop de valeurs NaN. Enlevons-les de notre analyse.


In [ ]:
series_indic_df = pd.DataFrame({'Catégorie': ['Clients',
                                              'Clients',
                                              'Clients',
                                              'Clients',
                                              'Clients',
                                              'Accès aux formations'],
                                'Indicateur': ['Population, ages 10-18, total',
                                               'Population, ages 15-24, total',
                                               'Gross enrolment ratio, primary to tertiary, gender parity index (GPI)',
                                               'Enrolment in tertiary education per 100,000 inhabitants, both sexes',
                                               'Population growth (annual %)',
                                               'Internet users (per 100 people)']})

### 2.3.2 Leur capacité à avoir accès aux formations

Cherchons les régions du monde satisfaisant le plus possible les indicateurs retenus.

Affichage des données accès aux formation par région du monde

In [ ]:
get_indics('Europe & Central Asia', '2014', 'Accès aux formations')


### 2.3.3 Leur pouvoir d'achat

Les deux chapitres 3.3 à 3.5 fournissent des indicateurs par pays, selon le pouvoir d'achat et les conditions d'activité de la start-up. Ils permettront de départager deux pays ou deux régions si le choix se pose.

Pondérisation des niveaux de richesse

In [ ]:
df_income = pd.DataFrame({'Income Group': ['High income: OECD',
                                           'High income: nonOECD',
                                           'Upper middle income',
                                           'Lower middle income',
                                           'Low income'],
                          'Income Group ponderation': [5,
                                                       4,
                                                       3,
                                                       2,
                                                       1]})
df_income.sort_values(by='Income Group ponderation', ascending=False)

### 2.3.4 Les conditions d'activité

Les deux chapitres 3.3 à 3.5 fournissent des indicateurs par pays, selon le pouvoir d'achat et les conditions d'activité de la start-up. Ils permettront de départager deux pays ou deux régions si le choix se pose.

Cherchons les régions du monde satisfaisant le plus possible les conditions retenus.

Pondérisation des conditions

In [ ]:
df_lending_categories = pd.DataFrame({'Lending category': ['Blend', 'IDA', 'IBRD'],
                                      'Lending category ponderation':[0.1, 0.2, 0.5]})
df_lending_categories.sort_values(by='Lending category ponderation', ascending=False)

In [ ]:
df_national_accounts = pd.DataFrame(
    {'System of National Accounts': ['Country uses the 1968 System of National Accounts methodology.',
                                     'Country uses the 1993 System of National Accounts methodology.',
                                     'Country uses the 2008 System of National Accounts methodology.'],
     'System of National Accounts ponderation': [1, 2, 3]})
df_national_accounts.sort_values(by='System of National Accounts ponderation', ascending=False)

In [ ]:
df_trade_system = pd.DataFrame({'System of trade':['Special trade system', 'General trade system', 'Other'],
                                'System of trade ponderation': [1, 2, 0]})
df_trade_system.sort_values(by='System of trade ponderation', ascending=False)

L'indicateur 'Source of most recent Income and expenditure data' n'a pas été pris en compte car il contient trop de valeurs uniques. On pourrait néanmoins utiliser les dates, mais leur traitement demandera un peu de temps. -> A faire si le reste de l'analyse est terminé.

### 2.3.5 Graphiques

Définissons une fonction qui nous permette d'ordonner une liste des **indicateurs** par **région**.

In [ ]:
for element in series_indic_df['Indicateur']:
    print(element)

In [ ]:
def indic_per_region(indic):
    df = df_data[df_data['Indicator Name'] == indic]
    df = df[df['Country Name'].isin(df_country['Region'].unique())]
    df = df[['Country Name', 'Indicator Name', '2020', '2025', '2030']]
    df = df.sort_values(by='2030', ascending=False)
    for region in df['Country Name']:
        x = ['2020', '2025', '2030']
        y = [df[df['Country Name']==region]['2020'],
             df[df['Country Name']==region]['2025'],
             df[df['Country Name']==region]['2030']]
        y = [float(element) for element in y]
        plt.grid(b=True)
        plt.plot(x, y, marker="o", markersize=12, label=region)
        plt.text(2.2, y[-2], region, fontsize=30)
    plt.rcParams["figure.figsize"] = (20,15)
    plt.show()
    return df, y

In [ ]:
for element in series_indic_df['Indicateur']:
    print(element)

In [ ]:
df_notnull = df_data[pd.notnull(df_data['2020'])]
df_filtered = df_notnull[['Country Name', 'Indicator Name', '2020']]
unique_indicators = df_filtered['Indicator Name'].unique()

In [ ]:
def indic_per_region_highincome(indic_):
    df_data = df_data[df_data['Income Group']=='High Income']
    return indic_per_region(indic_)

# 3. Conclusions

## 3.1 Réponse aux enjeux

## 3.2 Réponse à la problématique

Parmi les indicateurs identifiés au début de la pré-analyse comme pouvant potentiellement nous aider, certains sont soit remplis de valeurs NaN, soit avec uun nombre significatif de valeurs NaN. Nous les listons ci-dessous pour ne pas oublier qu'ils manqueront à l'analyse.

*   Gross enrolment ratio, primary to tertiary, gender parity index (GPI)
*   Enrolment in tertiary education per 100,000 inhabitants, both sexes
*   Unemployment, total (% of total labor force) (modeled ILO estimate)
*   Youth literacy rate, population 15-24 years, both sexes (%)
*   Personal computers (per 100 people)
*   Mortality rate, under-5 (per 1,000 live births)









Cela nous laisse les indicateurs suivants, en leur ajoutant un poids pour chacun :

In [ ]:
series_indic_list = {'Clients': ['Population, ages 10-18, total',
                                 'Population, ages 15-24, total',
                                 'Population growth (annual %)'],
                     'Accès aux formations': ['Internet users (per 100 people)']}
# C'est-à-dire, en utilisant cette fois-ci un DataFrame :
df_series_indic = pd.DataFrame({'Catégorie' : ['Clients',
                                               'Clients',
                                               'Clients',
                                               'Accès aux formations'],
                                'Indicateur': ['Population, ages 10-18, total',
                                               'Population, ages 15-24, total',
                                               'Population growth (annual %)',
                                               'Internet users (per 100 people)'],
                                'Ponderation': [2, 3, 4, 1]})
df_series_indic.sort_values(by='Ponderation', ascending=False)

Concernant les évaluateurs, nous avions vu que certains étaient inutilisables. Cela nous laisse les évaluateurs suivants :

In [ ]:
series_eval = {'Fiabilité des données' : ['Periodicity',
                                          'Aggregation method']}
df_series_eval = pd.DataFrame({'Catégorie': ['Fiabilité des données',
                                             'Fiabilité des données'],
                               'Evaluateur': ['Periodicity',
                                              'Aggregation method'],
                               'Ponderation': [1, 2]})
df_series_eval.sort_values(by='Ponderation', ascending=False)